# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vadso,70.0744,29.7487,-4.91,93,40,5.14,NO,1699832875
1,1,pudozh,61.8099,36.5345,2.75,98,100,2.68,RU,1699833064
2,2,port-aux-francais,-49.3500,70.2167,4.86,83,100,5.48,TF,1699832863
3,3,puerto ayora,-0.7393,-90.3518,24.75,90,45,1.79,EC,1699832837
4,4,papatowai,-46.5619,169.4708,12.60,94,100,0.81,NZ,1699832866


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display
# Configure the map
city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City",
)

# Display the map plot
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
cleaned_df = city_data_df[(city_data_df["Max Temp"]<27) & (city_data_df["Max Temp"]>21) & (city_data_df["Wind Speed"]<4.5) & (city_data_df["Cloudiness"]==0)]

# Drop any rows with null values
cleaned_df.dropna()

# Display sample data
cleaned_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
30,30,port mathurin,-19.6833,63.4167,24.92,81,0,2.03,MU,1699833089
46,46,tsiombe,-25.3000,45.4833,22.40,58,0,2.45,MG,1699833036
83,83,wickenburg,33.9686,-112.7296,21.88,21,0,3.74,US,1699833132
130,130,salalah,17.0151,54.0924,26.05,53,0,2.06,OM,1699833170
143,143,whangarei,-35.7317,174.3239,23.10,30,0,0.84,NZ,1699833182
203,203,bom jardim,-7.7958,-35.5872,21.12,81,0,2.30,BR,1699833232
267,267,grand baie,-20.0182,57.5802,21.92,72,0,1.34,MU,1699833286
325,325,cabedelo,-6.9811,-34.8339,26.37,76,0,4.16,BR,1699833334
348,348,al ghayzah,16.2079,52.1760,26.03,71,0,0.30,YE,1699833354
362,362,laguna,38.4210,-121.4238,23.71,32,0,2.57,US,1699833364


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = cleaned_df[["City","Country", "Lat", "Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
30,port mathurin,MU,-19.6833,63.4167,81,
46,tsiombe,MG,-25.3000,45.4833,58,
83,wickenburg,US,33.9686,-112.7296,21,
130,salalah,OM,17.0151,54.0924,53,
143,whangarei,NZ,-35.7317,174.3239,30,
203,bom jardim,BR,-7.7958,-35.5872,81,
267,grand baie,MU,-20.0182,57.5802,72,
325,cabedelo,BR,-6.9811,-34.8339,76,
348,al ghayzah,YE,16.2079,52.1760,71,
362,laguna,US,38.4210,-121.4238,32,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
limit = 20
params = {
    "categories": categories,
    "limit": limit,
    "apiKey": geoapify_key
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] =  f"proximity:{longitude},{latitude}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port mathurin - nearest hotel: Escale Vacances
tsiombe - nearest hotel: No hotel found
wickenburg - nearest hotel: Best Western Rancho Grande
salalah - nearest hotel: Muscat International Hotel
whangarei - nearest hotel: Grand Hotel
bom jardim - nearest hotel: Hotel Breno
grand baie - nearest hotel: Safari
cabedelo - nearest hotel: No hotel found
al ghayzah - nearest hotel: فندق تاج العرب
laguna - nearest hotel: Holiday Inn Express & Suites
zayed city - nearest hotel: L'Arabia hotel
pamandzi - nearest hotel: Gite Le Madina


,City,Country,Lat,Lng,Humidity,Hotel Name
30,port mathurin,MU,-19.6833,63.4167,81,Escale Vacances
46,tsiombe,MG,-25.3000,45.4833,58,No hotel found
83,wickenburg,US,33.9686,-112.7296,21,Best Western Rancho Grande
130,salalah,OM,17.0151,54.0924,53,Muscat International Hotel
143,whangarei,NZ,-35.7317,174.3239,30,Grand Hotel
203,bom jardim,BR,-7.7958,-35.5872,81,Hotel Breno
267,grand baie,MU,-20.0182,57.5802,72,Safari
325,cabedelo,BR,-6.9811,-34.8339,76,No hotel found
348,al ghayzah,YE,16.2079,52.1760,71,فندق تاج العرب
362,laguna,US,38.4210,-121.4238,32,Holiday Inn Express & Suites


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.8,
    color = "City",
    hover_cols = ["City","Hotel Name","Country"]
 
)

# Display the map plot
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)